In [119]:
# for basic data science
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import xmltodict
import json

# for downloading files off the internet
import urllib.request
import json
import time
import os
if os.path.basename(os.getcwd()) != 'food-pairing':
    os.chdir(os.path.dirname(os.getcwd()))

# for network graphs
from colour import Color
from matplotlib.collections import LineCollection
import networkx as nx
import mpld3

import requests
import json
from bs4 import BeautifulSoup
import html_to_json
from tqdm import tqdm

In [4]:
def flavordb_entity_url(x):
    return "https://foodb.ca/compounds/"+str(x)

In [112]:
failed = []

def read_food_from_url(x):
    r = requests.get(flavordb_entity_url(x))
    if r.status_code == 200:
        html_string = r.text
        o = xmltodict.parse(html_string)
        output = json.dumps(o)
        data = json.loads(output)
        ingredients = []
        quantities = []
        if (data['compound'])['foods'] is None:
            return [0], [0]
        foods = ((data['compound'])['foods']['food'])
        if type(foods) == list:
            for food in foods:
                ingredients.append(food['name'])
                try:
                    quantities.append(food['average_value'])
                except KeyError:
                    quantities.append(0)
        else:
            ingredients.append(foods['name'])
            try:
                quantities.append(foods['average_value'])
            except KeyError:
                quantities.append(0)
        return ingredients, quantities

    else:
        print(f"Failed to read {x}")
        failed.append(x)
        return [0], [0]

In [2]:
entities = pd.read_csv('foodb/Compound.csv')

names = entities['public_id'].values.tolist()

/var/folders/y5/sg4lr5xn7fv5ry3py7wvzj240000gn/T/ipykernel_90415/3469145740.py:1: DtypeWarning: Columns (3,4,5,6,12,13,14,15) have mixed types. Specify dtype option on import or set low_memory=False.
  entities = pd.read_csv('foodb/Compound.csv')


In [ ]:
df = pd.DataFrame(columns=['compound_id', 'foods', 'quantities'])

In [128]:
for name in tqdm(names[70355:]):
    foods, quantities = read_food_from_url(name)
    df.loc[len(df)] = [name, foods, quantities]

100%|██████████| 122/122 [02:20<00:00,  1.15s/it]


In [129]:
df.to_csv("compounds.csv", sep=';', index=False)

In [88]:
def read_json(x):
    r = requests.get(flavordb_entity_url(x))
    print(r.status_code)
    html_string = r.text
    o = xmltodict.parse(html_string)
    output = json.dumps(o)
    return json.loads(output)

In [104]:
data_long = read_json(names[10])
data_short = read_json(names[1])

200
200


In [106]:
data_long

{'compound': {'version': '1.0',
  'creation_date': '2010-04-08 22:04:25 UTC',
  'update_date': '2018-05-28 22:34:11 UTC',
  'accession': 'FDB000048',
  'name': 'p-Coumaroyl vitisin A',
  'description': 'A polyphenol compound found in foods of plant origin (PhenolExplorer).',
  'synonyms': {'synonym': ['Malvidin 3-(6-coumaroyl)-glucoside pyruvic acid',
    'Malvidin 3-(6-coumaroyl)-glucoside pyruvic adduct']},
  'chemical_formula': 'C35H31O16',
  'average_molecular_weight': '707.611',
  'monisotopic_moleculate_weight': '707.161209944',
  'iupac_name': None,
  'traditional_iupac': None,
  'cas_registry_number': '643-84-5',
  'smiles': 'COC1=CC(=CC(OC)=C1O)C1=[O+]C2=C3C(OC(=CC3=C1OC1O[C@H](COC(=O)\\C=C\\C3=CC=C(O)C=C3)[C@@H](O)[C@H](O)[C@H]1O)C(O)=O)=CC(O)=C2',
  'inchi': 'InChI=1S/C35H30O16/c1-45-22-9-16(10-23(46-2)28(22)39)32-33(19-13-24(34(43)44)48-20-11-18(37)12-21(49-32)27(19)20)51-35-31(42)30(41)29(40)25(50-35)14-47-26(38)8-5-15-3-6-17(36)7-4-15/h3-13,25,29-31,35,40-42H,14H2,1-2H3,(

In [105]:
print(type((data_long['compound'])['foods']['food']))
print(type((data_short['compound'])['foods']['food']))

TypeError: 'NoneType' object is not subscriptable

In [74]:
foods = ((data['compound'])['foods'])

In [76]:
len(foods)

for food in foods:
    print(food['name'])

TypeError: string indices must be integers, not 'str'